In [96]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
spark = sqlContext.sparkSession
!pip install plotly
import pandas as pd

In [97]:
import numpy as np
import matplotlib.pyplot as plt

In [98]:
df=spark.read.csv(r"/Users/aycaidilkaya/Desktop/total_crime_clean.csv",inferSchema=True)

In [99]:
df.count()

3007377

In [100]:
df.show(5)

+-----+--------------------+--------+-------------------+------+-----+-----+---+----+------------+-------------+
|  _c0|                 _c1|     _c2|                _c3|   _c4|  _c5|  _c6|_c7| _c8|         _c9|         _c10|
+-----+--------------------+--------+-------------------+------+-----+-----+---+----+------------+-------------+
|24417|02/23/2019 12:43:...|HOMICIDE|FIRST DEGREE MURDER|STREET|false| true| 15|2019|41.896721646|-87.759902047|
|24630|07/03/2019 05:30:...|HOMICIDE|FIRST DEGREE MURDER|STREET|false|false|  7|2019|41.768087791| -87.62662791|
|24705|08/24/2019 05:13:...|HOMICIDE|FIRST DEGREE MURDER|STREET|false|false| 11|2019|41.891753676|-87.719388155|
|24741|08/27/2019 06:40:...|HOMICIDE|FIRST DEGREE MURDER|STREET|false|false|  5|2019|41.678038746|-87.621771684|
|24751|09/16/2019 01:12:...|HOMICIDE|FIRST DEGREE MURDER|STREET|false|false| 11|2019|41.894539497|-87.713844747|
+-----+--------------------+--------+-------------------+------+-----+-----+---+----+-----------

In [101]:
df.cache()

DataFrame[_c0: int, _c1: string, _c2: string, _c3: string, _c4: string, _c5: boolean, _c6: boolean, _c7: int, _c8: int, _c9: double, _c10: double]

In [102]:
df=df.withColumnRenamed("_c0","ID").withColumnRenamed("_c1","Date").withColumnRenamed("_c2","PrimaryType")\
                                   .withColumnRenamed("_c3","Description").withColumnRenamed("_c4","LocationDes")\
                                   .withColumnRenamed("_c5","Arrest").withColumnRenamed("_c6","Domestic")\
                                   .withColumnRenamed("_c7","District").withColumnRenamed("_c8","Year")\
                                   .withColumnRenamed("_c9","Lat").withColumnRenamed("_c10","Long")

In [103]:
df.show(5)

+-----+--------------------+-----------+-------------------+-----------+------+--------+--------+----+------------+-------------+
|   ID|                Date|PrimaryType|        Description|LocationDes|Arrest|Domestic|District|Year|         Lat|         Long|
+-----+--------------------+-----------+-------------------+-----------+------+--------+--------+----+------------+-------------+
|24417|02/23/2019 12:43:...|   HOMICIDE|FIRST DEGREE MURDER|     STREET| false|    true|      15|2019|41.896721646|-87.759902047|
|24630|07/03/2019 05:30:...|   HOMICIDE|FIRST DEGREE MURDER|     STREET| false|   false|       7|2019|41.768087791| -87.62662791|
|24705|08/24/2019 05:13:...|   HOMICIDE|FIRST DEGREE MURDER|     STREET| false|   false|      11|2019|41.891753676|-87.719388155|
|24741|08/27/2019 06:40:...|   HOMICIDE|FIRST DEGREE MURDER|     STREET| false|   false|       5|2019|41.678038746|-87.621771684|
|24751|09/16/2019 01:12:...|   HOMICIDE|FIRST DEGREE MURDER|     STREET| false|   false|  

In [104]:
df.cache()

DataFrame[ID: int, Date: string, PrimaryType: string, Description: string, LocationDes: string, Arrest: boolean, Domestic: boolean, District: int, Year: int, Lat: double, Long: double]

***DATA ANALYSIS

In [105]:
#IMPORTS
from pyspark.sql import functions as F
import plotly.express as px
import plotly.graph_objects as go

***Making changes to the values and creating column that we will be using.

In [106]:
df = df.withColumn('Arrest',F.when(df['Arrest']=='false',0).otherwise(1))\
        .withColumn('Domestic',F.when(df['Domestic']=='false',0).otherwise(1))
df.show()

+-------+--------------------+--------------------+-------------------+-----------+------+--------+--------+----+------------+-------------+
|     ID|                Date|         PrimaryType|        Description|LocationDes|Arrest|Domestic|District|Year|         Lat|         Long|
+-------+--------------------+--------------------+-------------------+-----------+------+--------+--------+----+------------+-------------+
|  24417|02/23/2019 12:43:...|            HOMICIDE|FIRST DEGREE MURDER|     STREET|     0|       1|      15|2019|41.896721646|-87.759902047|
|  24630|07/03/2019 05:30:...|            HOMICIDE|FIRST DEGREE MURDER|     STREET|     0|       0|       7|2019|41.768087791| -87.62662791|
|  24705|08/24/2019 05:13:...|            HOMICIDE|FIRST DEGREE MURDER|     STREET|     0|       0|      11|2019|41.891753676|-87.719388155|
|  24741|08/27/2019 06:40:...|            HOMICIDE|FIRST DEGREE MURDER|     STREET|     0|       0|       5|2019|41.678038746|-87.621771684|
|  24751|09/1

In [107]:
split_cols=F.split(df["Date"]," ")
df=df.withColumn("Date_New",split_cols.getItem(0))
df=df.withColumn("Hours",split_cols.getItem(1))
df=df.withColumn("Hours",split_cols.getItem(1))
df=df.withColumn("AM/PM",split_cols.getItem(2))

In [108]:
df.select("Date_New","Hours","AM/PM").show(20)

+----------+--------+-----+
|  Date_New|   Hours|AM/PM|
+----------+--------+-----+
|02/23/2019|12:43:00|   AM|
|07/03/2019|05:30:00|   PM|
|08/24/2019|05:13:00|   PM|
|08/27/2019|06:40:00|   AM|
|09/16/2019|01:12:00|   AM|
|01/02/2010|12:15:00|   AM|
|06/09/2010|08:30:00|   PM|
|10/20/2010|09:00:00|   PM|
|12/11/2010|10:30:00|   AM|
|01/31/2011|10:30:00|   AM|
|06/01/2011|09:15:00|   AM|
|06/28/2011|09:25:00|   PM|
|07/16/2011|01:30:00|   PM|
|10/11/2011|03:00:00|   PM|
|03/22/2012|09:00:00|   PM|
|06/18/2012|07:45:00|   PM|
|06/20/2013|10:00:00|   AM|
|06/30/2013|11:30:00|   AM|
|07/02/2013|10:00:00|   PM|
|10/04/2013|12:24:00|   AM|
+----------+--------+-----+
only showing top 20 rows



In [109]:
#merging am/pm to hours for visualization
df = df.withColumn('HoursConcate', 
                    F.concat(F.col('Hours'),F.lit(' '), F.col('AM/PM')))
df.select("HoursConcate").show(5)

+------------+
|HoursConcate|
+------------+
| 12:43:00 AM|
| 05:30:00 PM|
| 05:13:00 PM|
| 06:40:00 AM|
| 01:12:00 AM|
+------------+
only showing top 5 rows



In [110]:
#splitting day month year because we will look them monthly
split_cols2=F.split(df["Date_New"],"/")
df=df.withColumn("Months",split_cols2.getItem(0))

***YEARLY

In [111]:
# YEARS CRIME COUNT
years_crime_count=df.groupBy("Year").agg(F.count(df["ID"]).alias("#crimes")).orderBy("Year",ascending=True)
years_crime_count.show()

+----+-------+
|Year|#crimes|
+----+-------+
|2010| 369792|
|2011| 350894|
|2012| 335082|
|2013| 306303|
|2014| 273579|
|2015| 257753|
|2016| 266458|
|2017| 264118|
|2018| 262792|
|2019| 256916|
|2020|  63690|
+----+-------+



In [112]:
def bar(df):
    fig=go.Figure(data=[go.Bar(x=df["Year"],y=df["#crimes"],text=df["#crimes"],textposition="auto",)])
    fig.update_traces(marker_color='rgb(0,206,209)', marker_line_color='rgb(124,252,0)',
                  marker_line_width=1.5, opacity=1)
    fig.update_layout(title='Total Crime Depending on Years', xaxis_title='Year', yaxis_title='Total Crime', plot_bgcolor ='beige', paper_bgcolor='whitesmoke')
    fig.show()
bar(years_crime_count.toPandas())


In [113]:
#YEARS CRIME TYPE FREQUENCY
years_type_fre=df.groupBy("Year","PrimaryType").agg(F.count(df["ID"]).alias("#crimes")).orderBy("Year","PrimaryType",ascending=True)
years_type_fre.show()


+----+--------------------+-------+
|Year|         PrimaryType|#crimes|
+----+--------------------+-------+
|2010|               ARSON|    522|
|2010|             ASSAULT|  21518|
|2010|             BATTERY|  65368|
|2010|            BURGLARY|  26392|
|2010| CRIM SEXUAL ASSAULT|   1331|
|2010|     CRIMINAL DAMAGE|  40613|
|2010|CRIMINAL SEXUAL A...|      3|
|2010|   CRIMINAL TRESPASS|   9397|
|2010|  DECEPTIVE PRACTICE|  12315|
|2010|            GAMBLING|    927|
|2010|            HOMICIDE|    438|
|2010|INTERFERENCE WITH...|    796|
|2010|        INTIMIDATION|    197|
|2010|          KIDNAPPING|    313|
|2010|LIQUOR LAW VIOLATION|    736|
|2010| MOTOR VEHICLE THEFT|  19021|
|2010|           NARCOTICS|  43380|
|2010|        NON-CRIMINAL|      1|
|2010|           OBSCENITY|     33|
|2010|OFFENSE INVOLVING...|   2506|
+----+--------------------+-------+
only showing top 20 rows



In [114]:
years_type_fre_fig=px.bar(years_type_fre.toPandas(),x="Year",y="#crimes",color="PrimaryType")
years_type_fre_fig.update_layout(title='Crime Type Frequency Depending on Years', xaxis_title='Year', yaxis_title='Total Crime', plot_bgcolor ='beige', paper_bgcolor='whitesmoke')

years_type_fre_fig.show()

In [115]:
#ARRESTED PER YEAR
arrest_crime_count=df.groupBy("Year").agg(F.count(df["Arrest"]==1).alias("Arrested")).sort("Year", ascending=True)
arrest_crime_count.show()


+----+--------+
|Year|Arrested|
+----+--------+
|2010|  369792|
|2011|  350894|
|2012|  335082|
|2013|  306303|
|2014|  273579|
|2015|  257753|
|2016|  266458|
|2017|  264118|
|2018|  262792|
|2019|  256916|
|2020|   63690|
+----+--------+



In [116]:
arrest_crime_count_fig=px.line(arrest_crime_count.toPandas(), x="Year", y="Arrested")
arrest_crime_count_fig.update_layout(title='Number of Arrests Through Years', xaxis_title='Year', yaxis_title='Number of Arrests', plot_bgcolor ='pink', paper_bgcolor='whitesmoke')
arrest_crime_count_fig.show()

In [117]:
#1. MURDER BY YEAR
murder_by_year=df.filter(df["PrimaryType"]=="HOMICIDE")
murder_by_year=murder_by_year.groupBy("Year").agg(F.count("ID").alias("#murders")).sort("Year",ascending=True)
murder_by_year.show()

+----+--------+
|Year|#murders|
+----+--------+
|2010|     438|
|2011|     438|
|2012|     515|
|2013|     429|
|2014|     426|
|2015|     502|
|2016|     788|
|2017|     674|
|2018|     600|
|2019|     501|
|2020|     155|
+----+--------+



In [118]:
def murder_bar(df):
    fig=go.Figure(data=[go.Bar(x=df["Year"],y=df["#murders"],text=df["#murders"],textposition="auto",)])
    fig.update_traces(marker_color='rgb(250,128,114)', marker_line_color='rgb(205,92,92)',
                  marker_line_width=1.5, opacity=1)
    fig.update_layout(title='Total # Homicides per Year', xaxis_title='Year', yaxis_title='Total Homicide', plot_bgcolor ='beige', paper_bgcolor='whitesmoke')
    fig.show()
murder_bar(murder_by_year.toPandas())
    

In [119]:
# THEFT BY YEAR
theft_by_year=df.filter(df["PrimaryType"]=="THEFT")
theft_by_year=theft_by_year.groupBy("Year").agg(F.count("ID").alias("#thefts")).sort("Year",ascending=True)
theft_by_year.show()

+----+-------+
|Year|#thefts|
+----+-------+
|2010|  76684|
|2011|  74853|
|2012|  75115|
|2013|  71457|
|2014|  61350|
|2015|  56696|
|2016|  61039|
|2017|  63584|
|2018|  64024|
|2019|  61612|
|2020|  14006|
+----+-------+



In [120]:
def theft_bar(df):
    fig=go.Figure(data=[go.Bar(x=df["Year"],y=df["#thefts"],text=df["#thefts"],textposition="auto",)])
    fig.update_traces(marker_color='rgb(0,250,154)', marker_line_color='rgb(46,139,87)',
                  marker_line_width=1.5, opacity=1)
    fig.update_layout(title='Total # Thefts per Year', xaxis_title='Year', yaxis_title='Total Theft', plot_bgcolor ='beige', paper_bgcolor='whitesmoke')
    fig.show()
theft_bar(theft_by_year.toPandas())

***MONTHLY

In [121]:
#MONTHS CRIME COUNT
months_crime_count=df.groupBy("Months").agg(F.count(df["ID"]).alias("#crimes")).sort("Months",ascending=True)
months_crime_count.show(20)

+------+-------+
|Months|#crimes|
+------+-------+
|    01| 248694|
|    02| 216906|
|    03| 257097|
|    04| 252001|
|    05| 265007|
|    06| 265503|
|    07| 275565|
|    08| 272932|
|    09| 252246|
|    10| 251482|
|    11| 228331|
|    12| 221613|
+------+-------+



In [122]:
months_crime_count_fig=px.line(months_crime_count.toPandas(),x="Months",y="#crimes")
months_crime_count_fig.update_layout(title='Total Crime Depending on Months', xaxis_title='Months', yaxis_title='Total Crime', plot_bgcolor ='beige', paper_bgcolor='whitesmoke')
months_crime_count_fig.show()

In [123]:
#MONTHS CRIME TYPE FREQUENCY
months_crime_fre=df.groupBy("Months","PrimaryType").agg(F.count(df["ID"]).alias("#crimes")).sort("Months",ascending=True)
months_crime_fre.show(20)

+------+--------------------+-------+
|Months|         PrimaryType|#crimes|
+------+--------------------+-------+
|    01|     CRIMINAL DAMAGE|  24573|
|    01|             ROBBERY|  10420|
|    01|             ASSAULT|  14797|
|    01|OFFENSE INVOLVING...|   2507|
|    01|            GAMBLING|     53|
|    01|               THEFT|  54603|
|    01|INTERFERENCE WITH...|    923|
|    01|   CRIMINAL TRESPASS|   6942|
|    01|       OTHER OFFENSE|  16875|
|    01|        INTIMIDATION|    120|
|    01| CRIM SEXUAL ASSAULT|   1242|
|    01|      NON - CRIMINAL|      2|
|    01|PUBLIC PEACE VIOL...|   1741|
|    01|             BATTERY|  42163|
|    01|        NON-CRIMINAL|     11|
|    01|LIQUOR LAW VIOLATION|    350|
|    01|           OBSCENITY|     46|
|    01|          KIDNAPPING|    195|
|    01|   HUMAN TRAFFICKING|     10|
|    01|OTHER NARCOTIC VI...|      3|
+------+--------------------+-------+
only showing top 20 rows



In [124]:
months_crime_fre_fig=px.bar(months_crime_fre.toPandas(),x="Months",y="#crimes",color="PrimaryType")
months_crime_fre_fig.update_layout(title='Crime Type Frequency Depending on Months', xaxis_title='Months', yaxis_title='Total Crime', plot_bgcolor ='beige', paper_bgcolor='whitesmoke')
months_crime_fre_fig.show()

***HOURS

In [125]:
#SPECIFIC HOURS CRIME COUNT
hours_crime_count=df.groupBy("Hours","AM/PM").agg(F.count("ID").alias("#crimes")).sort("#crimes",ascending=False).limit(50)
hours_crime_count.show()

+--------+-----+-------+
|   Hours|AM/PM|#crimes|
+--------+-----+-------+
|12:00:00|   PM|  72922|
|09:00:00|   AM|  69596|
|06:00:00|   PM|  57058|
|03:00:00|   PM|  54955|
|05:00:00|   PM|  54106|
|10:00:00|   PM|  53530|
|07:00:00|   PM|  53090|
|08:00:00|   PM|  52640|
|04:00:00|   PM|  51437|
|09:00:00|   PM|  50601|
|02:00:00|   PM|  49894|
|10:00:00|   AM|  48274|
|01:00:00|   PM|  45352|
|08:00:00|   AM|  44442|
|11:00:00|   AM|  39392|
|11:00:00|   PM|  38264|
|12:00:00|   AM|  35392|
|12:01:00|   AM|  33823|
|01:00:00|   AM|  29961|
|03:30:00|   PM|  27559|
+--------+-----+-------+
only showing top 20 rows



In [126]:
hours_crime_count_fig=px.bar(hours_crime_count.toPandas(),x="Hours",y="#crimes",color="AM/PM")
hours_crime_count_fig.update_layout(title='Total Crime on Specific Hours', xaxis_title='Hours', yaxis_title='Total Crime', plot_bgcolor ='beige', paper_bgcolor='whitesmoke')
hours_crime_count_fig.show()

In [127]:
#AM/PM-CRIME COUNT
amPm_crime_count=df.groupBy("AM/PM").agg(F.count("ID").alias("#crimes")).orderBy("AM/PM")
amPm_crime_count.show()

+-----+-------+
|AM/PM|#crimes|
+-----+-------+
|   AM|1109370|
|   PM|1898007|
+-----+-------+



In [128]:
def am_pm_bar(df):
    fig=go.Figure(data=[go.Bar(x=df["AM/PM"],y=df["#crimes"],text=df["#crimes"],textposition="auto",)])
    fig.update_traces(marker_color='rgb(205,92,92)', marker_line_color='rgb(128,0,0)',
                  marker_line_width=1.5, opacity=1)
    fig.update_layout(title='Total Crime Depending on AM/PM', xaxis_title='AM/PM', yaxis_title='Total Crime', plot_bgcolor ='beige', paper_bgcolor='whitesmoke')
    fig.show()
am_pm_bar(amPm_crime_count.toPandas())




***LOCATION

In [129]:
#DISTRICT - CRIME COUNT
dis_crime_count=df.groupBy("District").agg(F.count("ID").alias("Total Crime")).sort("District",ascending=True)
dis_crime_count.show()

+--------+-----------+
|District|Total Crime|
+--------+-----------+
|       1|     133566|
|       2|     127450|
|       3|     150665|
|       4|     173436|
|       5|     134421|
|       6|     181457|
|       7|     171911|
|       8|     199159|
|       9|     143256|
|      10|     137083|
|      11|     204893|
|      12|     144341|
|      14|     110445|
|      15|     128800|
|      16|     100055|
|      17|      86341|
|      18|     140864|
|      19|     134675|
|      20|      49973|
|      22|      97931|
+--------+-----------+
only showing top 20 rows



In [130]:
def dis_bar(df):
    fig=go.Figure(data=[go.Bar(x=df["District"],y=df["Total Crime"],text=df["Total Crime"],textposition="auto",)])
    fig.update_traces(marker_color='rgb(135,206,250)', marker_line_color='rgb(30,144,255)',
                  marker_line_width=1.5, opacity=1)
    fig.update_layout(title='Total Crime Depending on Distrcits', xaxis_title='District', yaxis_title='Total Crime', plot_bgcolor ='beige', paper_bgcolor='whitesmoke')
    fig.show()

dis_bar(dis_crime_count.toPandas())




In [131]:
#DISTRICT CRIME TYPE FREQUENCY
dis_crime_type=df.rollup("District","PrimaryType").agg(F.count("PrimaryType").alias("#Crime")).orderBy("District").sort("District",ascending=True).dropna()
dis_crime_type.show(25)

+--------+--------------------+------+
|District|         PrimaryType|#Crime|
+--------+--------------------+------+
|       1|   CRIMINAL TRESPASS|  6276|
|       1|CONCEALED CARRY L...|    12|
|       1|             ASSAULT|  6325|
|       1|            BURGLARY|  1998|
|       1|     CRIMINAL DAMAGE|  6961|
|       1|          KIDNAPPING|    44|
|       1|        PROSTITUTION|   247|
|       1|      NON - CRIMINAL|     1|
|       1|OTHER NARCOTIC VI...|    14|
|       1|    PUBLIC INDECENCY|    21|
|       1| CRIM SEXUAL ASSAULT|   456|
|       1|LIQUOR LAW VIOLATION|   131|
|       1|           OBSCENITY|    21|
|       1|CRIMINAL SEXUAL A...|    18|
|       1|OFFENSE INVOLVING...|   304|
|       1|  DECEPTIVE PRACTICE| 15963|
|       1|            HOMICIDE|    38|
|       1|INTERFERENCE WITH...|   186|
|       1|   HUMAN TRAFFICKING|     5|
|       1| MOTOR VEHICLE THEFT|  2523|
|       1|        NON-CRIMINAL|    15|
|       1|         SEX OFFENSE|   440|
|       1|            STA

In [132]:
fig2 = px.bar(dis_crime_type.toPandas(), x='District', y='#Crime',color='PrimaryType', width=1000, height=700)
fig2.update_layout(title='Crime Type Frequency Depends on District', xaxis_title='District', yaxis_title='Amount of Crime', plot_bgcolor ='pink', paper_bgcolor='whitesmoke')
fig2.show()

In [133]:
#COORDINATES CRIME COUNT MAPPING
cor_crime_count=df.groupBy("Lat","Long").agg(F.count('ID').alias('Total Crime')).sort("Total Crime",ascending=False).limit(100)
cor_crime_count.show()

+------------+-------------+-----------+
|         Lat|         Long|Total Crime|
+------------+-------------+-----------+
|41.883500187|-87.627876698|       4511|
|41.754592961|-87.741528537|       3838|
|41.897895128|-87.624096605|       2859|
|41.979006297|-87.906463155|       2322|
|41.909664252|-87.742728815|       2178|
|41.868541914|-87.639235361|       1955|
|41.896888586|-87.628203192|       1926|
|41.868180939|-87.709271389|       1924|
|41.976290414|-87.905227221|       1849|
|41.742710224|-87.634088181|       1820|
|41.885487535|-87.726422045|       1603|
|41.976762981|-87.900983721|       1361|
|41.899410159|-87.624131266|       1251|
|41.882394062|-87.627844798|       1223|
|41.891990384|-87.611461502|       1177|
|42.019399237|-87.675049485|       1168|
|41.891694878|-87.626155832|       1152|
| 41.88171846|-87.627760426|       1145|
|41.750940757|-87.625185222|       1103|
|41.788987036| -87.74147999|       1036|
+------------+-------------+-----------+
only showing top

In [134]:

def plot_map(dataframe,focusOn):
 fig = go.Figure(data=go.Scattergeo(lon = dataframe['Long'], lat = dataframe['Lat'], mode = 'markers', marker_color = dataframe[focusOn], marker = dict(size = 9, opacity = 0.8, reversescale = True, autocolorscale = True, symbol = 'square', line = dict(width=2, color='white'),colorscale = 'teal', cmin = 0, color = dataframe[focusOn], cmax = dataframe[focusOn].max(), colorbar_title="The range of {}".format(focusOn))))
 fig.update_layout(margin=dict(l=20, r=20, t=20, b=20),title = 'Plot')
 fig.show()
plot_map(cor_crime_count.toPandas(),"Total Crime")

In [135]:
#4.Coordinates-Crime Type Frequency

In [136]:
# LOCATION DESCRIPTION -CRIME COUNT
locDes_crime_count=df.groupBy("LocationDes").agg(F.count("ID").alias("#crimes")).sort("#crimes",ascending=False).limit(15)
locDes_crime_count.show()

+--------------------+-------+
|         LocationDes|#crimes|
+--------------------+-------+
|              STREET| 693587|
|           RESIDENCE| 491209|
|           APARTMENT| 380005|
|            SIDEWALK| 311073|
|               OTHER| 112684|
|PARKING LOT/GARAG...|  86353|
|               ALLEY|  64142|
|  SMALL RETAIL STORE|  63234|
|RESIDENTIAL YARD ...|  59118|
|          RESTAURANT|  58140|
|VEHICLE NON-COMME...|  51859|
|    RESIDENCE-GARAGE|  51830|
|RESIDENCE PORCH/H...|  51792|
|SCHOOL, PUBLIC, B...|  51127|
|    DEPARTMENT STORE|  44141|
+--------------------+-------+



In [137]:
def loc_bar(df):
    fig=go.Figure(data=[go.Bar(x=df["LocationDes"],y=df["#crimes"],text=df["#crimes"],textposition="auto",)])
    fig.update_traces(marker_color='rgb(138,43,226)', marker_line_color='rgb(25,25,112)',
                  marker_line_width=1.5, opacity=1)
    fig.update_layout(title='Crime Type Frequency Depending on Location Description', xaxis_title='Year', yaxis_title='Total Crime', plot_bgcolor ='beige', paper_bgcolor='whitesmoke')
    fig.show()
loc_bar(locDes_crime_count.toPandas())